In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
BATCH_SIZE = 128
EPOCHS = 10

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))])

# Import MNIST dataset

In [4]:
trainset = torchvision.datasets.MNIST(root = './data',
                                                        train = True,
                                                        download = True,
                                                        transform = transform)

trainloader = torch.utils.data.DataLoader(trainset,
                                                             batch_size = BATCH_SIZE,
                                                             shuffle = True,
                                                             num_workers = 2)

In [5]:
testset = torchvision.datasets.MNIST(root = './data',
                                                      train = False,
                                                       download = True,
                                                        transform = transform)

testloader = torch.utils.data.DataLoader(testset, 
                                                            batch_size = BATCH_SIZE,
                                                            shuffle = False,
                                                            num_workers = 2)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, 
                                            out_channels = 32, 
                                            kernel_size = 3,
                                            stride = 1) 
        
        self.conv2 = nn.Conv2d(in_channels = 32,
                                            out_channels = 64,
                                            kernel_size = 3,
                                            stride = 1) 
        
        self.pool = nn.MaxPool2d(kernel_size = 2, 
                                               stride = 2) 
        
        self.dropout1 = nn.Dropout2d(p = 0.5)
        self.fc1 = nn.Linear(in_features = 12 * 12 * 64, out_features = 128)
        self.dropout2 = nn.Dropout2d(p = 0.5)
        self.fc2 = nn.Linear(128, 10) # There are 10 classes [0,1,2,3,4,5,6,7,8,9]
        
    
    def forward(self, x):
        x = F.relu(self.conv1(x)) #(28, 28, 1) -> (26, 26, 32)
        x = self.pool(F.relu(self.conv2(x))) #(26, 26, 32) -> (24, 24, 64) -> (12, 12, 64)
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64) #reshape from (12, 12, 64) to (1, 9216)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [7]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001)

# Train

In [8]:
for epoch in range(EPOCHS):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        optimizer.zero_grad()
        
        #forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i +1, running_loss / 100))
            running_loss = 0.0
print("Finished Training")

[1,   100] loss: 2.300
[1,   200] loss: 2.281
[1,   300] loss: 2.264
[1,   400] loss: 2.240
[2,   100] loss: 2.184
[2,   200] loss: 2.135
[2,   300] loss: 2.080
[2,   400] loss: 2.006
[3,   100] loss: 1.840
[3,   200] loss: 1.717
[3,   300] loss: 1.587
[3,   400] loss: 1.466
[4,   100] loss: 1.262
[4,   200] loss: 1.168
[4,   300] loss: 1.073
[4,   400] loss: 0.984
[5,   100] loss: 0.899
[5,   200] loss: 0.836
[5,   300] loss: 0.798
[5,   400] loss: 0.757
[6,   100] loss: 0.711
[6,   200] loss: 0.692
[6,   300] loss: 0.673
[6,   400] loss: 0.648
[7,   100] loss: 0.622
[7,   200] loss: 0.615
[7,   300] loss: 0.592
[7,   400] loss: 0.572
[8,   100] loss: 0.564
[8,   200] loss: 0.556
[8,   300] loss: 0.541
[8,   400] loss: 0.539
[9,   100] loss: 0.523
[9,   200] loss: 0.519
[9,   300] loss: 0.497
[9,   400] loss: 0.496
[10,   100] loss: 0.479
[10,   200] loss: 0.491
[10,   300] loss: 0.472
[10,   400] loss: 0.475
Finished Training


In [9]:
correct = 0
total = 0

with torch.no_grad():
    for (images, labels) in testloader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        

print('Accuracy of the network: %d %%' % (
    100 * correct / total))

Accuracy of the network: 87 %
